In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import re
from decimal import Decimal, InvalidOperation

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
raw_web = pd.read_csv('WEB_ORDERS - WEB_ORDERS.csv')
raw_leads = pd.read_csv('LEADS_FUNNEL - LEADS_FUNNEL.csv')
raw_channels = pd.read_csv('CHANNELS - CHANNELS.csv')


In [4]:
raw_channels.head(150)

,CAMPAIGN_ID,CAMPAIGN_NAME,CAMPAIGN_PERIOD_BUDGET_CATEGORY,CHANNEL_3,CHANNEL_4,CHANNEL_5
0,120211151098440066,fb_ie_landing_prospecting_pos-lite_poslite-cr_bau_alwayson,alwayson,fb,fb prospecting,fb prospecting landing
1,391180926,bing-search_gb_generic_prospecting_self-serve_cr,NaN,bing-search,bing-search prospecting,bing-search prospecting generic
2,21020541104,max_gb_prospecting,unknown,max,max prospecting,max prospecting generic
3,16335200574,search-shopping_gb_brand_prospecting_self-serve_cr,NaN,search-shopping,search-shopping prospecting,search-shopping prospecting brand
4,20774689177,search_gb_generic_prospecting_sales-led_pos_promo_alwayson_pos-broad,alwayson,search,search prospecting,search prospecting generic
...,...,...,...,...,...,...
115,20589784451,search_gb_brand_prospecting_sales-led_pos_bau_alwayson_point-of-sale,alwayson,search,search prospecting,search prospecting brand
116,20924620040,demandgen_ gb_generic_mix_sales-led_bundle_promo_flight_mix-ot,flight,demand gen,demandgen mix,demandgen mix generic
117,12598739708,search_gb_brand_remarketing_self-serve_cr,NaN,search,search remarketing,search remarketing brand
118,21127704718,search_gb_generic_highvolume vbb4,unknown,search,search prospecting,search prospecting generic


In [5]:
import numpy as np
import pandas as pd

cid = 120211151482640066
cols = ['CHANNEL_3', 'CHANNEL_4', 'CHANNEL_5']  

df = raw_channels.copy()

df[cols] = df[cols].replace({'unknown': pd.NA, 'Unknown': pd.NA, 'UNKNOWN': pd.NA})


lookup = (
    df.groupby('CAMPAIGN_ID', as_index=True)[cols]
      .agg(lambda s: s.dropna().iloc[0] if s.notna().any() else pd.NA)
)

mask = df['CAMPAIGN_ID'].eq(cid)
df.loc[mask, cols] = df.loc[mask, cols].fillna(lookup.loc[cid])


df[cols] = df[cols].fillna('unknown')


df = df.sort_values(cols).drop_duplicates(subset=['CAMPAIGN_ID'], keep='first')

raw_channels = df


In [6]:
raw_channels = raw_channels.drop_duplicates().reset_index(drop=True)

In [7]:
#def clean_campaign_names(series: pd.Series) -> pd.Series:
  #  return (series
   #         .astype('string')
   #         .str.lower()
   #         .str.replace(' ', '', regex=False) 
    #       )


def fix_campaign_id(series: pd.Series) -> pd.Series:
    s = series.astype("string").str.strip()
    s = s.replace({"<NA>": pd.NA, "nan": pd.NA, "None": pd.NA, "": pd.NA})

    def _one(x):
        if pd.isna(x):
            return pd.NA
        t = str(x).strip()

       
        if t.endswith(".000000"):
            t = t[:-7]
        if t.endswith(".0"):
            t = t[:-2]

    
        if "e" in t.lower():
            try:
                d = Decimal(t)
                t = format(d.quantize(Decimal("1")), "f")
            except (InvalidOperation, ValueError):
                return pd.NA

  
        digits = "".join(ch for ch in t if ch.isdigit())
        return digits if digits else pd.NA

    return s.map(_one).astype("string")



def clean_strings(x):
    if pd.isna(x):
        return pd.NA

    t = str(x).lower().strip()

   
    t = re.sub(r"\s+", " ", t)

    t = re.sub(r"p\s*r\s*o\s*s\s*p\s*e\s*c\s*t\s*i\s*n\s*g", "prospecting", t)
    t = re.sub(r"s\s*h\s*o\s*p\s*p\s*i\s*n\s*g", "shopping", t)
    t = re.sub(r"l\s*a\s*n\s*d\s*i\s*n\s*g", "landing", t)
    t = re.sub(r"d\s*e\s*m\s*a\s*n\s*d\s*[-_ ]*\s*g\s*e\s*n", "demandgen", t)

   
    t = re.sub(r"p\s*o\s*s\s*[-_ ]*\s*l\s*i\s*t\s*e", "pos-lite", t)

    
    t = re.sub(r"\s*-\s*", "-", t)
    t = re.sub(r"\s*_\s*", "_", t)

   
    t = t.replace(" ", "_")


    
    t = re.sub(r"bing_search_+", "bing-search_", t)
    t = re.sub(r"search_shopping_+", "search-shopping_", t)

   
    t = re.sub(r"_+", "_", t)
    t = t.strip("_")

    return t
    
    

In [8]:
channels = raw_channels.copy()
raw_web_copy = raw_web.copy()
leads = raw_leads.copy()

In [9]:
raw_web_copy['CAMPAIGN_ID'] = fix_campaign_id(raw_web_copy['CAMPAIGN_ID'])
leads['CAMPAIGN_ID'] = fix_campaign_id(leads['CAMPAIGN_ID'])
channels['CAMPAIGN_ID'] = fix_campaign_id(channels['CAMPAIGN_ID'])

In [10]:
leads['CAMPAIGN_NAME'] = leads['CAMPAIGN_NAME'].apply(clean_strings)
leads['CHANNEL_3'] = leads['CHANNEL_3'].apply(clean_strings)
leads['CHANNEL_4'] = leads['CHANNEL_4'].apply(clean_strings)
leads['CHANNEL_5'] = leads['CHANNEL_5'].apply(clean_strings)

channels['CAMPAIGN_NAME'] = channels['CAMPAIGN_NAME'].apply(clean_strings)
channels['CHANNEL_3'] = channels['CHANNEL_3'].apply(clean_strings)
channels['CHANNEL_4'] = channels['CHANNEL_4'].apply(clean_strings)
channels['CHANNEL_5'] = channels['CHANNEL_5'].apply(clean_strings)


In [11]:
web = raw_web_copy.merge(channels, how="left", on="CAMPAIGN_ID")

In [12]:
leads_copy = leads.copy()
channels_df = channels.copy()


channels_df = (
    channels_df.sort_values("CAMPAIGN_ID")
    .groupby("CAMPAIGN_ID", as_index=False)
    .last()
)


leads_copy = leads_copy.set_index("CAMPAIGN_ID")
channels_df = channels_df.set_index("CAMPAIGN_ID")


leads_copy.update(channels_df)


leads_filled = leads_copy.reset_index()



In [ ]:
#PRODUCT is not included because all Products are POS Lite
#cross-checking do pairs campaign_id - campaign_name are unique?

cols = ["CHANNEL_3", "CHANNEL_4", "CHANNEL_5"]

dfs = []

for col in cols:
    tmp = (
        leads
        .groupby("CAMPAIGN_ID", dropna=False)
        .agg(
            col_values=(col, lambda s: sorted(s.fillna("<<NaN>>").astype(str).unique())),
            col_nunique=(col, lambda s: s.fillna("<<NaN>>").nunique()),
            campaign_names=("CAMPAIGN_NAME", lambda s: sorted(s.fillna("<<NaN>>").astype(str).unique())),
            campaign_name_nunique=("CAMPAIGN_NAME", lambda s: s.fillna("<<NaN>>").nunique()),
        )
        .reset_index()
        .assign(profile_col=col)
    )
    dfs.append(tmp)

dict_columns_leads = (
    pd.concat(dfs, ignore_index=True)
      .sort_values(["col_nunique", "campaign_name_nunique"], ascending=False)
)

In [ ]:
# PRODUCT is not included because all Products are POS Lite

cols = ["CHANNEL_3", "CHANNEL_4", "CHANNEL_5"]

dfs = []

for col in cols:
    tmp = (
        raw_channels
        .assign(
            CAMPAIGN_NAME=raw_channels["CAMPAIGN_NAME"].fillna("<<NaN>>").astype(str),
            CAMPAIGN_ID=raw_channels["CAMPAIGN_ID"].fillna("<<NaN>>").astype(str),
        )
        .groupby("CAMPAIGN_NAME", dropna=False)
        .agg(
           
            col_values=(col, lambda s: sorted(s.fillna("<<NaN>>").astype(str).unique())),
            col_nunique=(col, lambda s: s.fillna("<<NaN>>").nunique()),

            
            campaign_id_values=("CAMPAIGN_ID", lambda s: sorted(s.unique())),
            campaign_id_nunique=("CAMPAIGN_ID", "nunique"),
        )
        .reset_index()
        .assign(profile_col=col)
    )
    dfs.append(tmp)

dict_columns_channels_name = (
    pd.concat(dfs, ignore_index=True)
      .sort_values(["campaign_id_nunique", "col_nunique"], ascending=False)
)


In [19]:
import pandas as pd

result = (
    channels
    .assign(
        CAMPAIGN_NAME=raw_channels["CAMPAIGN_NAME"].fillna("<<NaN>>").astype(str),
        CAMPAIGN_ID=raw_channels["CAMPAIGN_ID"].fillna("<<NaN>>").astype(str),
    )
    .groupby("CAMPAIGN_NAME", dropna=False)
    .agg(
        campaign_id_values=("CAMPAIGN_ID", lambda s: sorted(s.unique())),
        campaign_id_nunique=("CAMPAIGN_ID", "nunique"),
    )
    .reset_index()
    .sort_values("campaign_id_nunique", ascending=False)
)


In [20]:
result

,CAMPAIGN_NAME,campaign_id_values,campaign_id_nunique
0,search-shopping_es_brand+gen_prospecting_self-serve,[21250282073],1
1,<<NaN>>,[20991759296],1
32,search-shopping_ie_brand+gen_prospecting_self-serve_cr,[16792023515],1
33,search-shopping_ie_generic_prospecting_pos-lite_poslite-cr_bau_alwayson,[21182413864],1
34,search_ch_brand_prospecting_self-serve_cr_de,[16736661269],1
35,search_ch_brand_prospecting_self-serve_cr_fr,[21296828618],1
36,search_ch_branding_prospecting_self-serve_french,[21209555471],1
37,search_ch_generic-fr,[21296853803],1
38,search_ch_generic_prospecting_self-serve_cr_de,[16820221869],1
39,search_es_brand+gen_prospecting_self-serve,[21250221815],1


In [21]:
test_leads = leads.copy()
test_web = web.copy()

In [22]:
# Assumption: each Campaign_id has a unique Campaign_Name because: * due to investigation of Channels dataset found that id-name pairs are unique
#                                                                  * no contradictions were found in Leads and Web datasets
# Here happens mapping mising Campaign Name and Campaign_id in bidirectional way. 
# How it works: 2 dictionaries: 1.1 -> dict(id:name) - here is finding missing pairs accross web and leads datasets.
#                               1.2 -> dict(name:id) - bidirectional
#                               2 ->dict(id:CAMPAIGN_PERIOD_BUDGET_CATEGORY, COUNTRY_CODE, CHANNEL_3,CHANNEL_4, CHANNEL_5) - map rest missing values

KEY_ID = "CAMPAIGN_ID"
KEY_NAME = "CAMPAIGN_NAME"
ATTR_COLS = (
    "CAMPAIGN_PERIOD_BUDGET_CATEGORY",
    "COUNTRY_CODE",
    "CHANNEL_3",
    "CHANNEL_4",
    "CHANNEL_5",
)

NA_STRINGS = {"<<nan>>", "nan", "none", "<na>", "", "unknown"}


def normalize_na_vectorized(series: pd.Series) -> pd.Series:
    """Vectorized NA normalization - no strip() needed if data is clean"""
    s = series.astype("string")
    mask = s.str.lower().isin(NA_STRINGS) | s.isna()
    return s.mask(mask, pd.NA)


def mode_nonnull_fast(series: pd.Series):
    """Optimized mode calculation"""
    s = series.dropna()
    if len(s) == 0:
        return pd.NA
    
    vc = s.value_counts()
    if len(vc) == 0:
        return pd.NA
    
    max_count = vc.iloc[0]
    modes = vc[vc == max_count].index
    
    return modes.min() if len(modes) > 0 else pd.NA


def normalize_dataframe_inplace(df: pd.DataFrame, cols) -> None:
    """Normalize in-place to avoid copies"""
    for c in cols:
        if c in df.columns:
            df[c] = normalize_na_vectorized(df[c])


def build_campaign_maps_optimized(df_a: pd.DataFrame, df_b: pd.DataFrame):
    """Build both maps in single pass"""
    
    pairs_list = []
    for df in (df_a, df_b):
        if KEY_ID in df.columns and KEY_NAME in df.columns:
            # Don't normalize here - assume already normalized
            tmp = df[[KEY_ID, KEY_NAME]].dropna()
            if len(tmp) > 0:
                pairs_list.append(tmp)
    
    if not pairs_list:
        return {}, {}
    
    pairs = pd.concat(pairs_list, ignore_index=True).drop_duplicates()
    
    id_counts = pairs.groupby(KEY_ID)[KEY_NAME].nunique()
    name_counts = pairs.groupby(KEY_NAME)[KEY_ID].nunique()
    
    valid_ids = id_counts[id_counts == 1].index
    valid_names = name_counts[name_counts == 1].index
    
    id_pairs = pairs[pairs[KEY_ID].isin(valid_ids)].drop_duplicates(subset=[KEY_ID])
    name_pairs = pairs[pairs[KEY_NAME].isin(valid_names)].drop_duplicates(subset=[KEY_NAME])
    
    id_to_name = dict(zip(id_pairs[KEY_ID], id_pairs[KEY_NAME]))
    name_to_id = dict(zip(name_pairs[KEY_NAME], name_pairs[KEY_ID]))
    
    return id_to_name, name_to_id


def apply_campaign_maps_inplace(df: pd.DataFrame, id_to_name: dict, name_to_id: dict) -> None:
    """Apply maps in-place"""
    if KEY_ID in df.columns and KEY_NAME in df.columns:
        df[KEY_NAME] = df[KEY_NAME].fillna(df[KEY_ID].map(id_to_name))
        df[KEY_ID] = df[KEY_ID].fillna(df[KEY_NAME].map(name_to_id))


def build_attribute_reference_optimized(df_a: pd.DataFrame, df_b: pd.DataFrame, 
                                        key_col: str, attr_cols=ATTR_COLS):
    """Optimized attribute reference building"""
    
    cols_needed = [key_col] + [c for c in attr_cols if c in df_a.columns or c in df_b.columns]
    
    dfs_to_concat = []
    for df in (df_a, df_b):
        if key_col in df.columns:
            cols_present = [c for c in cols_needed if c in df.columns]
            if cols_present:
                dfs_to_concat.append(df[cols_present])
    
    if not dfs_to_concat:
        return pd.DataFrame(columns=[key_col])
    
    all_df = pd.concat(dfs_to_concat, ignore_index=True)
    present_attrs = [c for c in attr_cols if c in all_df.columns]
    
    if not present_attrs:
        return pd.DataFrame(columns=[key_col])
    
   
    ref = (
        all_df.dropna(subset=[key_col])
              .groupby(key_col, dropna=False)[present_attrs]
              .agg(mode_nonnull_fast)
              .reset_index()
    )
    
    return ref


def fill_from_reference_inplace(df: pd.DataFrame, ref: pd.DataFrame, 
                                key_col: str, attr_cols=ATTR_COLS) -> None:
    """Fill attributes in-place using reference"""
    if key_col not in df.columns or key_col not in ref.columns:
        return
    
    present_attrs = [c for c in attr_cols if c in df.columns and c in ref.columns]
    if not present_attrs:
        return
    
 
    if len(ref) < 10000:
        for attr in present_attrs:
            attr_map = ref.set_index(key_col)[attr].to_dict()
            df[attr] = df[attr].fillna(df[key_col].map(attr_map))
    else:
        merged = df[[key_col]].merge(
            ref[[key_col] + present_attrs], 
            on=key_col, 
            how="left", 
            suffixes=("", "_ref")
        )
        
        for attr in present_attrs:
            df[attr] = df[attr].fillna(merged[f"{attr}_ref"])


cols_to_norm = [KEY_ID, KEY_NAME, *ATTR_COLS]
normalize_dataframe_inplace(test_leads, cols_to_norm)
normalize_dataframe_inplace(test_web, cols_to_norm)

id_to_name, name_to_id = build_campaign_maps_optimized(test_leads, test_web)


apply_campaign_maps_inplace(test_leads, id_to_name, name_to_id)
apply_campaign_maps_inplace(test_web, id_to_name, name_to_id)

ref_by_id = build_attribute_reference_optimized(test_leads, test_web, 
                                                key_col=KEY_ID, attr_cols=ATTR_COLS)
ref_by_name = build_attribute_reference_optimized(test_leads, test_web, 
                                                  key_col=KEY_NAME, attr_cols=ATTR_COLS)

fill_from_reference_inplace(test_leads, ref_by_id, key_col=KEY_ID, attr_cols=ATTR_COLS)
fill_from_reference_inplace(test_web, ref_by_id, key_col=KEY_ID, attr_cols=ATTR_COLS)

fill_from_reference_inplace(test_leads, ref_by_name, key_col=KEY_NAME, attr_cols=ATTR_COLS)
fill_from_reference_inplace(test_web, ref_by_name, key_col=KEY_NAME, attr_cols=ATTR_COLS)

present_attr_cols = [c for c in ATTR_COLS if c in ref_by_id.columns]
campaign_id_to_attrs = ref_by_id.set_index(KEY_ID)[present_attr_cols].to_dict(orient="index")

In [23]:
def derive_channel_viz(
    df: pd.DataFrame,
    name_col: str = "CAMPAIGN_NAME",
    test_flag_col: str = "test",
    channel_col: str = "CHANNEL",
    overwrite: bool = False,
) -> None:
    """
    Derive campaign fields in-place (no copy).
    Modifies the DataFrame directly.
    """
    
    # Derive test flag
    if name_col in df.columns:
        derived_test = df[name_col].astype("string").str.lower().str.contains("test", na=False, regex=False)
        if test_flag_col not in df.columns:
            df[test_flag_col] = derived_test
        elif overwrite:
            df[test_flag_col] = derived_test
        else:
            mask = df[test_flag_col].isna()
            df.loc[mask, test_flag_col] = derived_test.loc[mask]
    
    # Channel mapping
    if 'CHANNEL_3' in df.columns:
        channel_mapping = {
            'fb': 'Facebook', 'bing-search': 'Bing Search',
            'bing-shopping': 'Bing Shopping', 'search': 'Search',
            'search-shopping': 'Search Shopping', 'max': 'Max',
        }
        derived_channel = df['CHANNEL_3'].map(channel_mapping).fillna('Unknown')
        if channel_col not in df.columns:
            df[channel_col] = derived_channel
        elif overwrite:
            df[channel_col] = derived_channel
        else:
            mask = df[channel_col].isna()
            df.loc[mask, channel_col] = derived_channel.loc[mask]

    if 'CHANNEL_5' in df.columns:
       
        split_channel = df['CHANNEL_5'].astype('string').str.rsplit('_', n=2, expand=True)
        
       
        if split_channel.shape[1] >= 3:
            df['CHANNEL_6'] = split_channel[1]  
            df['CHANNEL_7'] = split_channel[2]  
        else:
            df['CHANNEL_6'] = pd.NA
            df['CHANNEL_7'] = pd.NA




In [24]:
def add_country_name(df: pd.DataFrame, country_col: str = "COUNTRY_CODE", out_col: str = "COUNTRY") -> None:
    mapping = {
        "ES": "Spain",
        "GB": "United Kingdom",
        "IE": "Ireland",
        "CH": "Switzerland",
    }
    s = df[country_col].astype("string") if country_col in df.columns else pd.Series([pd.NA]*len(df))
    df[out_col] = s.map(mapping).fillna("Unknown")
    return df




def add_calendar_columns(df: pd.DataFrame, date_col: str = "DATE") -> None:
    dt = pd.to_datetime(df[date_col], errors="coerce")

    df["YEAR"] = dt.dt.year
    df["MONTH"] = dt.dt.month
    df["MONTH_NAME"] = dt.dt.month_name()
    df["DAY"] = dt.dt.day
    df["DAY_OF_WEEK"] = dt.dt.dayofweek + 1          # 1=Mon ... 7=Sun 
    df["DAY_NAME"] = dt.dt.day_name()
    df["DAY_OF_YEAR"] = dt.dt.dayofyear
    df["WEEK"] = dt.dt.isocalendar().week.astype("Int64")
    df["ISO_YEAR"] = dt.dt.isocalendar().year.astype("Int64")
    df["QUARTER"] = dt.dt.quarter

    df["WEEK_START"] = dt.dt.to_period("W-MON").dt.start_time

    return df

In [25]:
# WEB source
derive_channel_viz(test_web, name_col="CAMPAIGN_NAME")
add_country_name(test_web, country_col="COUNTRY_CODE", out_col="COUNTRY_NAME")
add_calendar_columns(test_web, date_col="DATE")

# LEADS source
derive_channel_viz(test_leads, name_col="CAMPAIGN_NAME")
add_country_name(test_leads, country_col="COUNTRY_CODE", out_col="COUNTRY_NAME")
add_calendar_columns(test_leads, date_col="DATE")


,DATE,CURRENCY,COUNTRY_CODE,CAMPAIGN_ID,CAMPAIGN_NAME,PRODUCT,CHANNEL_3,CHANNEL_4,CHANNEL_5,TOTAL_IMPRESSIONS,TOTAL_CLICKS,TOTAL_SPEND,TOTAL_LEADS,TOTAL_FAKE_LEADS,TOTAL_SQLS,TOTAL_MEETING_DONE,TOTAL_SIGNED_LEADS,TOTAL_POS_LITE_DEALS,test,CHANNEL,CHANNEL_6,CHANNEL_7,COUNTRY_NAME,YEAR,MONTH,MONTH_NAME,DAY,DAY_OF_WEEK,DAY_NAME,DAY_OF_YEAR,WEEK,ISO_YEAR,QUARTER,WEEK_START
0,2022-03-21,EUR,ES,23861494371910170,fb_es_pos-lite_prospecting_landing_promo,pos,fb,fb_prospecting,fb_prospecting_generic,NaN,NaN,NaN,7.0,6.0,0.0,0.0,7.0,0.0,False,Facebook,prospecting,generic,Spain,2022,3,March,21,1,Monday,80,12,2022,1,2022-03-15
1,2022-03-18,EUR,ES,23861494371910170,fb_es_pos-lite_prospecting_landing_promo,unknown,fb,fb_prospecting,fb_prospecting_generic,10.0,0.0,0.0,2.0,1.0,11.0,10.0,0.0,3.0,False,Facebook,prospecting,generic,Spain,2022,3,March,18,5,Friday,77,11,2022,1,2022-03-15
2,2022-05-13,EUR,ES,120211151482640060,fb_es_landing_prospecting_pos-lite_pos-lite-cr_bau_alwayson,poslite-cr,fb,fb_prospecting,fb_prospecting_landing,14810.0,62.0,80.0,0.0,0.0,10.0,9.0,2.0,4.0,False,Facebook,prospecting,landing,Spain,2022,5,May,13,5,Friday,133,19,2022,2,2022-05-10
3,2022-02-04,EUR,ES,23861494371910170,fb_es_pos-lite_prospecting_landing_promo,unknown,fb,fb_prospecting,fb_prospecting_generic,0.0,0.0,0.0,7.0,6.0,3.0,2.0,3.0,0.0,False,Facebook,prospecting,generic,Spain,2022,2,February,4,5,Friday,35,5,2022,1,2022-02-01
4,2022-03-06,EUR,ES,23861494371910170,fb_es_pos-lite_prospecting_landing_promo,unknown,fb,fb_prospecting,fb_prospecting_generic,8796.0,32.0,74.0,6.0,5.0,11.0,10.0,0.0,7.0,False,Facebook,prospecting,generic,Spain,2022,3,March,6,7,Sunday,65,9,2022,1,2022-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3731,2022-05-13,EUR,IE,21182413864,search-shopping_ie_generic_prospecting_pos-lite_pos-lite-cr_bau_alwayson,poslite-cr,search-shopping,search-shopping_prospecting,search-shopping_prospecting_generic,3616.0,40.0,333.0,13.0,11.0,11.0,10.0,0.0,7.0,False,Search Shopping,prospecting,generic,Ireland,2022,5,May,13,5,Friday,133,19,2022,2,2022-05-10
3732,2022-05-18,EUR,ES,120211151482640060,fb_es_landing_prospecting_pos-lite_pos-lite-cr_bau_alwayson,poslite-cr,fb,fb_prospecting,fb_prospecting_landing,36338.0,150.0,224.0,14.0,12.0,0.0,0.0,7.0,3.0,False,Facebook,prospecting,landing,Spain,2022,5,May,18,3,Wednesday,138,20,2022,2,2022-05-17
3733,2022-04-21,EUR,CH,21178981397,search-shopping_ch_generic_prospecting_pos-lite_pos-lite-cr_bau_alwayson,poslite-cr,search-shopping,search-shopping_prospecting,search-shopping_prospecting_generic,886.0,4.0,3.0,9.0,8.0,11.0,10.0,4.0,4.0,False,Search Shopping,prospecting,generic,Switzerland,2022,4,April,21,4,Thursday,111,16,2022,2,2022-04-19
3734,2022-05-28,EUR,CH,120211150575530060,fb_ch_landing_prospecting_pos-lite_pos-lite_bau_alwayson_selfserve_convgoaltest-purchasecr,pos-lite,fb,fb_prospecting,fb_prospecting_landing,9600.0,28.0,75.0,0.0,0.0,0.0,0.0,3.0,6.0,True,Facebook,prospecting,landing,Switzerland,2022,5,May,28,6,Saturday,148,21,2022,2,2022-05-24


In [87]:

def prep_web_long(web_df: pd.DataFrame) -> pd.DataFrame:
    df = web_df.copy()
    df.columns = df.columns.str.strip().str.lower()

    #  dims
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    else:
        raise ValueError("Web: no column DATE")

    if "campaign_id" not in df.columns:
        raise ValueError("Web: no CAMPAIGN_ID")

    # pathway 
    df["pathway"] = "Website (Self-Service)"

    # spend
    spend_col = "total_spend_eur" if "total_spend_eur" in df.columns else ("total_spend" if "total_spend" in df.columns else None)
    df["spend_value"] = df[spend_col] if spend_col else np.nan

    metric_map = {
        "nb_of_sessions": "sessions",
        "nb_of_signups": "signups",
        "nb_of_orders": "orders",
        "nb_of_poslite_items_ordered": "poslite_items_ordered",
    }
    metric_cols = [c for c in metric_map.keys() if c in df.columns]

    long_df = df.melt(
            id_vars = [
        "date","country_code","campaign_id","pathway","spend_value",
        "country_name","year","month","week","day_of_year","week_start",
        "channel","channel_3","channel_4","channel_5","channel_6","channel_7",
        "test"
    ],
        value_vars=metric_cols,
        var_name="funnel_stage",
        value_name="metric_value"
    )
    long_df["funnel_stage"] = long_df["funnel_stage"].map(metric_map)
    long_df["metric_value"] = pd.to_numeric(long_df["metric_value"], errors="coerce")

    return long_df



In [88]:
def prep_leads_long(leads_df: pd.DataFrame) -> pd.DataFrame:
    df = leads_df.copy()
    df.columns = df.columns.str.strip().str.lower()

    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # pathway
    df["pathway"] = "Contact Form (Sales-Assisted)"

    # spend
    spend_col = "total_spend" if "total_spend" in df.columns else ("total_spend_eur" if "total_spend_eur" in df.columns else None)
    df["spend_value"] = df[spend_col] if spend_col else np.nan

    metric_map = {
        "total_impressions": "impressions",
        "total_clicks": "clicks",
        "total_leads": "leads",
        "total_sqls": "sqls",
        "total_meeting_done": "meeting_done",
        "total_signed_leads": "signed_leads",
        "total_pos_lite_deals": "pos_lite_deals",
        "total_fake_leads": "fake_leads",
    }
    metric_cols = [c for c in metric_map.keys() if c in df.columns]

    long_df = df.melt(
        id_vars = [
        "date","country_code","campaign_id","pathway","spend_value",
        "country_name","year","month","week","day_of_year","week_start",
        "channel","channel_3","channel_4","channel_5","channel_6","channel_7",
        "test"
    ],
        value_vars=metric_cols,
        var_name="funnel_stage",
        value_name="metric_value"
    )
    long_df["funnel_stage"] = long_df["funnel_stage"].map(metric_map)
    long_df["metric_value"] = pd.to_numeric(long_df["metric_value"], errors="coerce")

    return long_df


In [89]:
def long_to_wide(long_df: pd.DataFrame) -> pd.DataFrame:
    index_cols = [
        "date","country_code","campaign_id","pathway","spend_value",
        "country_name","year","month","week","day_of_year","week_start",
        "channel","channel_3","channel_4","channel_5","channel_6","channel_7",
        "test"
    ]
    index_cols = [c for c in index_cols if c in long_df.columns]

    wide = (
        long_df
        .pivot_table(
            index=index_cols,
            columns="funnel_stage",
            values="metric_value",
            aggfunc="sum"
        )
        .reset_index()
    )
    wide.columns.name = None
    return wide


In [90]:
web_long   = prep_web_long(test_web)       
leads_long = prep_leads_long(test_leads)    

web_wide   = long_to_wide(web_long)
leads_wide = long_to_wide(leads_long)

In [94]:
web_long.to_csv("web_long.csv", index=False, sep=";")
leads_long.to_csv("leads_long.csv", index=False, sep=";")

web_wide.to_csv("web_wide.csv", index=False, sep=";")
leads_wide.to_csv("leads_wide.csv", index=False, sep=";")



all_long = pd.concat([web_long, leads_long], ignore_index=True)
all_wide = pd.concat([web_wide, leads_wide], ignore_index=True)

all_long.to_csv("all_long.csv", index=False, sep=";")
all_wide.to_csv("all_wide.csv", index=False, sep=";")


In [ ]:
import pandas as pd

#selecting max row by spending


group_cols = [ "country_code", "campaign_id", "date","pathway", "channel"]


metric_cols = [
    "spend_value",
    "orders",
    "poslite_items_ordered",
    "sessions",
    "signups",
    "clicks",
    "fake_leads",
    "impressions",
    "leads",
    "meeting_done",
    "pos_lite_deals",
    "signed_leads",
    "sqls",
]

dim_cols = [
    "country_name",
    "channel_3",
    "channel_4",
    "channel_5",
    "channel_6",
    "channel_7",
    "test",
]


group_cols = [c for c in group_cols if c in all_wide.columns]
metric_cols = [c for c in metric_cols if c in all_wide.columns]
dim_cols = [c for c in dim_cols if c in all_wide.columns]


required_group = ["date", "country_code", "campaign_id", "pathway", "channel"]
missing_group = set(required_group) - set(group_cols)
if missing_group:
    raise KeyError(f"Missing required group columns in all_wide: {sorted(missing_group)}")

def first_nonnull(s: pd.Series):
    s2 = s.dropna()
    return s2.iloc[0] if len(s2) else pd.NA


agg_dict = {c: "max" for c in metric_cols}
agg_dict.update({c: first_nonnull for c in dim_cols})

tableau_df = (
    all_wide
        .groupby(group_cols, as_index=False)
        .agg(agg_dict)
)

tableau_df = tableau_df[group_cols + dim_cols + metric_cols]



   country_code         campaign_id       date                        pathway  \
0            CH  120211150575530060 2022-05-07  Contact Form (Sales-Assisted)   
1            CH  120211150575530060 2022-05-07         Website (Self-Service)   
2            CH  120211150575530060 2022-05-08  Contact Form (Sales-Assisted)   
3            CH  120211150575530060 2022-05-08         Website (Self-Service)   
4            CH  120211150575530060 2022-05-09  Contact Form (Sales-Assisted)   
5            CH  120211150575530060 2022-05-09         Website (Self-Service)   
6            CH  120211150575530060 2022-05-10  Contact Form (Sales-Assisted)   
7            CH  120211150575530060 2022-05-10         Website (Self-Service)   
8            CH  120211150575530060 2022-05-11  Contact Form (Sales-Assisted)   
9            CH  120211150575530060 2022-05-11         Website (Self-Service)   
10           CH  120211150575530060 2022-05-12  Contact Form (Sales-Assisted)   
11           CH  12021115057

In [ ]:
import pandas as pd




group_cols = ["date", "country_code", "campaign_id", "pathway", "channel"]


metric_cols = [
    "spend_value",
    "orders",
    "poslite_items_ordered",
    "sessions",
    "signups",
    "clicks",
    "fake_leads",
    "impressions",
    "leads",
    "meeting_done",
    "pos_lite_deals",
    "signed_leads",
    "sqls",
]


dim_cols = [
    "country_name",
    "channel_3",
    "channel_4",
    "channel_5",
    "channel_6",
    "channel_7",
    "test",
]


group_cols = [c for c in group_cols if c in all_wide.columns]
metric_cols = [c for c in metric_cols if c in all_wide.columns]
dim_cols = [c for c in dim_cols if c in all_wide.columns]


required_group = ["date", "country_code", "campaign_id", "pathway", "channel"]
missing_group = set(required_group) - set(group_cols)
if missing_group:
    raise KeyError(f"Missing required group columns in all_wide: {sorted(missing_group)}")

def first_nonnull(s: pd.Series):
    s2 = s.dropna()
    return s2.iloc[0] if len(s2) else pd.NA


agg_dict = {c: "max" for c in metric_cols}
agg_dict.update({c: first_nonnull for c in dim_cols})

tableau_df = (
    all_wide
        .groupby(group_cols, as_index=False)
        .agg(agg_dict)
)


tableau_df = tableau_df[group_cols + dim_cols + metric_cols]


duplicate_check_cols = ["campaign_id", "date", "pathway"]
duplicates = tableau_df[tableau_df.duplicated(subset=duplicate_check_cols, keep=False)]

if len(duplicates) > 0:
    print(f"⚠️  WARNING: Found {len(duplicates)} duplicate rows!")
    print(f"\nDuplicate combinations of {duplicate_check_cols}:")
    print(duplicates.sort_values(duplicate_check_cols))
    
  
    print("\n🔍 Duplicate group counts:")
    print(duplicates.groupby(duplicate_check_cols).size().sort_values(ascending=False))
    

else:
    print(f" No duplicates found on {duplicate_check_cols}")


assert not tableau_df.duplicated(subset=duplicate_check_cols).any(), \
    f"Duplicates found on {duplicate_check_cols}!"

# 7) view / export
print("\n" + "="*60)
print(tableau_df.head(20))
tableau_df.to_csv("tableau_marketing_perf.csv", index=False)

✅ No duplicates found on ['campaign_id', 'date', 'pathway']

         date country_code        campaign_id                        pathway  \
0  2022-01-01           ES        17300002700         Website (Self-Service)   
1  2022-01-01           ES        17558850680         Website (Self-Service)   
2  2022-01-01           ES        19678441368         Website (Self-Service)   
3  2022-01-01           ES        19735056861         Website (Self-Service)   
4  2022-01-01           ES        20499484310         Website (Self-Service)   
5  2022-01-01           ES  23861494371910170  Contact Form (Sales-Assisted)   
6  2022-01-01           GB        15846715387         Website (Self-Service)   
7  2022-01-01           GB        16335200574         Website (Self-Service)   
8  2022-01-01           GB        16335307917         Website (Self-Service)   
9  2022-01-01           GB        18461205812         Website (Self-Service)   
10 2022-01-01           GB        20589784451         Websi

In [167]:
all_wide[
    (all_wide["campaign_id"] == "6476940146") &
    (all_wide["date"] == "2022-02-15")
]


,date,country_code,campaign_id,pathway,spend_value,country_name,year,month,week,day_of_year,week_start,channel,channel_3,channel_4,channel_5,channel_6,channel_7,test,orders,poslite_items_ordered,sessions,signups,clicks,fake_leads,impressions,leads,meeting_done,pos_lite_deals,signed_leads,sqls
1143,2022-02-15,GB,6476940146,Website (Self-Service),1996.455994,United Kingdom,2022.0,2.0,7,46.0,2022-02-15,Search,search,search_prospecting,search_prospecting_brand,prospecting,brand,False,1.0,0.0,299.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
